In [37]:
import spacy 
import json 
import random 

def load_data(file):
    """
    takes json file location and 
    returns python object representation of that json file 
    """
    with open(file,"r", encoding='utf-8') as f: 
        data = json.load(f) 
    return data 



def save_data(file, data): 
    with open(file,"w", encoding='utf-8') as f: 
        json.dump(data,f, indent=4)  

In [38]:
TRAIN_DATA = load_data('hp_training_data.json') 

In [4]:
def train_spacy(data, iterations): 
    """
    iterations: epochs on the training dataset 
    """
    TRAIN_DATA = data 
    nlp = spacy.blank('en')
    if "ner" not in nlp.pipe_names: 
        ner = nlp.create_pipe("ner") 
        nlp.add_pipe(ner, last=True) 




In [56]:
from tqdm.notebook import tqdm

TRAIN_DATA = load_data('hp_training_data.json') 
iterations = 30
TRAIN_DATA2 = []
for val in TRAIN_DATA:
    if val:
        TRAIN_DATA2.append(val)
nlp = spacy.blank('en')

if "ner" not in nlp.pipe_names: 
    #ner = nlp.create_pipe("ner") 
    ner = nlp.add_pipe('ner', last=True) 

for a, annotations in TRAIN_DATA2: 
    for ent in annotations.get('entities'): 
        ner.add_label(ent[2])
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
if len(other_pipes) == 0:
    optimizer = nlp.begin_training()
    for itn in tqdm(range(iterations)):
        print('starting iteration' + str(itn)) 
        random.shuffle(TRAIN_DATA2)
        losses = {}
        from spacy.training.example import Example

        for batch in spacy.util.minibatch(TRAIN_DATA2, size=50):
            for text, annotations in batch:
                # create Example
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                # Update the model
                nlp.update([example], losses=losses, drop=0.3, sgd = optimizer)
nlp



#from spacy.training.example import Example

#for batch in spacy.util.minibatch(TRAIN_DATA2, size=2):
#    for text, annotations in batch:
#        # create Example
#        doc = nlp.make_doc(text)
#        example = Example.from_dict(doc, annotations)
#        # Update the model
#        nlp.update([example], losses=losses, drop=0.3, sgd = optimizer)

#nlp.pipe_names

  0%|          | 0/30 [00:00<?, ?it/s]

starting iteration0
starting iteration1
starting iteration2
starting iteration3
starting iteration4
starting iteration5
starting iteration6
starting iteration7
starting iteration8
starting iteration9
starting iteration10
starting iteration11
starting iteration12
starting iteration13
starting iteration14
starting iteration15
starting iteration16
starting iteration17
starting iteration18
starting iteration19
starting iteration20
starting iteration21
starting iteration22
starting iteration23
starting iteration24
starting iteration25
starting iteration26
starting iteration27
starting iteration28
starting iteration29


In [57]:
nlp.to_disk('hp_ner_model')

In [58]:
def train_spacy(data, iterations):
    from tqdm.notebook import tqdm 
    import random
    TRAIN_DATA = data 
    TRAIN_DATA2 = []
    for val  in TRAIN_DATA:
        if val:
            TRAIN_DATA2.append(val)
    
    nlp = spacy.blank('en')
    if "ner" not in nlp.pipe_names:
        ner = nlp.add_pipe('ner', last=True) 
    
    for _, annotations in TRAIN_DATA2:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2]) 
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner'] 
    if len(other_pipes) == 0:
        optimizer = nlp.begin_training() 
        for itn in tqdm(range(iterations)):
            print('starting iteration:' + str(itn))
            random.shuffle(TRAIN_DATA2)
            losses = {} 
            from spacy.training.example import Example 

            for batch in spacy.util.minibatch(TRAIN_DATA2, size = 256):
                for text, annotations in batch: 
                    doc = nlp.make_doc(text)
                    example = Example.from_dict(doc, annotations)
                    nlp.update([example], losses=losses, drop = 0.3, sgd = optimizer)
    return nlp 